In [ ]:
from dataloader import get_data
from runner import NLBRunner
from model import VRNN

import torch
from datetime import datetime
import os


dataset_name = 'mc_maze_large'
phase = 'val'
bin_size = 5
data_path = '/scratch/gilbreth/akamsali/Research/Makin/000138/sub-Jenkins'
# Extract data
dataset, train_dict, eval_dict, training_input, training_output, eval_input = get_data(data_path, dataset_name, phase, bin_size)

# Train/val split and convert to Torch tensors
num_train = int(round(training_input.shape[0] * 0.75))
train_input = torch.Tensor(training_input[:num_train])
train_output = torch.Tensor(training_output[:num_train])
val_input = torch.Tensor(training_input[num_train:])
val_output = torch.Tensor(training_output[num_train:])
eval_input = torch.Tensor(eval_input)


RUN_NAME = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '_model'
RUN_DIR = './runs/'
if not os.path.isdir(RUN_DIR):
    os.mkdir(RUN_DIR)

DROPOUT = 0
L2_WEIGHT = 5e-7
LR_INIT = 1.5e-2
CD_RATIO = 0.27
HIDDEN_DIM = 40
USE_GPU = True
MAX_GPUS = 1

runner = NLBRunner(
    model_init=VRNN,
    model_cfg={'x_dim': train_input.shape[2], 'h_dim': HIDDEN_DIM, 'z_dim': HIDDEN_DIM, 'out_dim': train_output.shape[2], 'n_layers': 1, 'dropout': DROPOUT},
    data=(train_input, train_output, val_input, val_output, eval_input),
    train_cfg={'lr': LR_INIT, 'alpha': L2_WEIGHT, 'cd_ratio': CD_RATIO},
    use_gpu=USE_GPU,
    num_gpus=MAX_GPUS,
)